# Check sparse implementation

In [1]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from inference_model import (
    sum_contacts, sum_messages, vector_csr, 
    get_infection_probas_dmp, get_infection_probas_mean_field
)
from sir_model import get_infection_probas

def generate_list_csr(N, nnz):
    i = np.random.choice(N, nnz, replace=False)
    j = np.random.choice(N, nnz, replace=False)
    values = np.random.rand(nnz)
    A_list = list(zip(i, j, values))
    A_csr = coo_matrix((values, (i, j)), shape=(N, N)).tocsr()
    return A_list, A_csr

N, nnz = 500, 3
transmissions_list, transmissions = generate_list_csr(N, nnz)
contacts_list = [(i, j) for (i, j, _) in transmissions_list]
contacts = (transmissions!=0)

# get_infection_probas (sir)

In [2]:
def naive_get_infection_probas(states, transmissions):
    """
    - states[i] = state of i
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = 1 - prod_{j: state==I} [1 - lambda_ij]
    """
    infected = (states == 1)
    N = len(states)
    infection_probas = np.zeros(N)
    for i in range(N):
        rates = np.array([
            rate for i0, j, rate in transmissions
            if i0 == i and infected[j]
        ])
        infection_probas[i] = 1 - np.prod(1 - rates)
    return infection_probas

In [3]:
states = np.ones(N)

In [4]:
p_old = naive_get_infection_probas(states, transmissions_list)
p_new = get_infection_probas(states, transmissions)
assert np.allclose(p_old, p_new)
print(p_old[p_old>0])
print(p_new[p_new>0])

[0.12497695 0.8762881  0.85213022]
[0.12497695 0.8762881  0.85213022]


In [5]:
%timeit p_old = naive_get_infection_probas(states, transmissions_list)

5.12 ms ± 219 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit p_new = get_infection_probas(states, transmissions)

159 µs ± 9.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# get_infection_probas_mean_field

In [7]:
i = np.random.choice(N, nnz, replace=False)
j = np.random.choice(N, nnz, replace=False)
values = np.random.rand(nnz)

In [8]:
# coo then csr is faster
%timeit coo_matrix((values, (i, j)), shape=(N, N)).tocsr()

141 µs ± 7.44 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
%timeit csr_matrix((values, (i, j)), shape=(N, N))

226 µs ± 25.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
def naive_get_infection_probas_mean_field(probas, transmissions):
    """
    - probas[i,s] = P_s^i(t)
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = sum_j lambda_ij P_I^j(t)
    """
    N = probas.shape[0]
    infection_probas = np.zeros(N)
    for i in range(N):
        rates = np.array([
            probas[j, 1]*rate
            for i0, j, rate in transmissions if i0 == i
        ])
        infection_probas[i] = rates.sum()
    return infection_probas

In [11]:
probas = np.ones((N, 3))/3.

In [12]:
p_old = naive_get_infection_probas_mean_field(probas, transmissions_list)
p_new = get_infection_probas_mean_field(probas, transmissions)
assert np.allclose(p_old, p_new)
print(p_old[p_old>0])
print(p_new[p_new>0])

[0.04165898 0.29209603 0.28404341]
[0.04165898 0.29209603 0.28404341]


In [13]:
%timeit p_old = naive_get_infection_probas_mean_field(probas, transmissions_list)

2.92 ms ± 42.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit p_new = get_infection_probas_mean_field(probas, transmissions)

7.52 µs ± 50.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# sum_contacts

In [15]:
def naive_sum_contacts(A, contacts):
    """
    - A : list of i, k, A_ik
    - contacts : list of i, k contacts
    - a_i = sum_{k in i} A_ik  = sum_k A_ik contacts_ik
    """
    a = np.zeros(N)
    for i in range(N):
        a[i] = sum(val for i_, k, val in A if (i_, k) in contacts and i_==i)
    return a

In [16]:
a_old = naive_sum_contacts(transmissions_list, contacts_list)
a_new = sum_contacts(transmissions, contacts)
assert np.allclose(a_old, a_new)
print(a_old[a_old>0])
print(a_new[a_new>0])

[0.12497695 0.8762881  0.85213022]
[0.12497695 0.8762881  0.85213022]


In [17]:
%timeit a_old = naive_sum_contacts(transmissions_list, contacts_list)

903 µs ± 24.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
%timeit a_new = sum_contacts(transmissions, contacts)

162 µs ± 6.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# sum_messages

In [19]:
def naive_sum_messages(A, contacts):
    """
    - A = list of i, k, A_ik
    - contacts = list of i, k contacts
    - B_ij = sum_{k in j\i} A_jk  = sum_k A_jk contacts_jk (k != i)
    """
    B = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            B[i, j] = sum(
                val for j_, k, val in A 
                if (j_, k) in contacts and i!=k and j_==j
            )
    return B

In [20]:
unequal = 1 - np.identity(N)
B_old = naive_sum_messages(transmissions_list, contacts_list)
B_new = sum_messages(transmissions, contacts)
assert np.allclose(B_new.toarray(), B_old)

In [21]:
%timeit B_old = naive_sum_messages(transmissions_list, contacts_list)

623 ms ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit B_new = sum_messages(transmissions, contacts)

3.84 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# vector_csr

In [23]:
v = np.zeros(10)
v[:3] = [4,5,6]
A=vector_csr(v)
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        assert A[i, j] == v[j]
print("A.nnz", A.nnz)

A.nnz 30


# Gotcha !

In scipy sparse matrix, opposite to numpy, operator $*$ is matrix multiplication, not element wise multiply ! Use `.multiply()` for element-wise multiply.

In [24]:
N = 100
nnz = 3
v = np.random.rand(N)
A_list, A = generate_list_csr(N, nnz)
B_list, B = generate_list_csr(N, nnz)

In [25]:
# Av = A.multiply(v) computes Av_ij = A_ij v_j
Av = A.multiply(v).tocsr()
for i in range(N):
    for j in range(N):
        assert Av[i, j] == A[i, j]*v[j]

In [26]:
AB = A*B
AB_array = A.toarray() @ B.toarray()
assert np.allclose(AB.toarray(), AB_array)

In [27]:
AB = A.multiply(B)
AB_array = A.toarray() * B.toarray()
assert np.allclose(AB.toarray(), AB_array)